# Generating Qubit Hamiltonians

In [1]:
from utility import *

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ atoms. 

In [2]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h2)

-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099508 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099503 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109117 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109117 [Z3]


Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [3]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=4, n_electrons=2, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.5310513494337641 [] +
0.1967905834854702 [X0] +
-0.5350572998841725 [Z0]


We can verify that this new Hamiltonian indeed includes the ground state. 

In [4]:
print("The ground state energy:")
obtain_PES('h2', [1], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The ground state energy:
E = -1.1011503301329564 Eh

The eigenvalues in the effective Hamiltonian: 
 [-1.10115031  0.03904763]


# Let's generate the H4 Hamiltonian

In [18]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h4 = get_qubit_hamiltonian(mol='h4', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h4)

-0.24347573739328132 [] +
-0.033078463224609334 [X0 X1 Y2 Y3] +
-0.03536722155262414 [X0 X1 Y4 Y5] +
-0.0193796150075185 [X0 X1 Y6 Y7] +
0.033078463224609334 [X0 Y1 Y2 X3] +
0.03536722155262414 [X0 Y1 Y4 X5] +
0.0193796150075185 [X0 Y1 Y6 X7] +
0.03355113608569815 [X0 Z1 X2 X3 Z4 Z5 Z6 X7] +
0.03355113608569815 [X0 Z1 X2 Y3 Z4 Z5 Z6 Y7] +
-0.03364279215972457 [X0 Z1 Z2 Z3 X4 X5 Z6 X7] +
-0.03364279215972457 [X0 Z1 Z2 Z3 X4 Y5 Z6 Y7] +
-0.009390698830211788 [X0 Z1 Z2 Z3 Z4 Z5 X6] +
-0.0005277563218391576 [X0 Z1 Z2 Z3 Z4 Z5 X6 Z7] +
0.019481400318485782 [X0 Z1 Z2 Z3 Z4 X6] +
-0.01416139184123879 [X0 Z1 Z2 Z3 Z5 X6] +
-0.019381467627564326 [X0 Z1 Z2 Z4 Z5 X6] +
0.014169668458133822 [X0 Z1 Z3 Z4 Z5 X6] +
0.000613366410994583 [X0 Z2 Z3 Z4 Z5 X6] +
0.033078463224609334 [Y0 X1 X2 Y3] +
0.03536722155262414 [Y0 X1 X4 Y5] +
0.0193796150075185 [Y0 X1 X6 Y7] +
-0.033078463224609334 [Y0 Y1 X2 X3] +
-0.03536722155262414 [Y0 Y1 X4 X5] +
-0.0193796150075185 [Y0 Y1 X6 X7] +
0.03355113608569815 [Y0 Z1 Y

In [19]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h4, n_spin_orbitals=8, n_electrons=4, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 0.053452480564595034 [] +
0.033078463224609334 [X0] +
0.005545747540744297 [X0 X1] +
0.03355113608569815 [X0 Z1 X2 Z3] +
-0.033078463224609334 [X0 Z1 Z2] +
-0.03355113608569815 [X0 Z1 Z2 X3] +
-0.033078463224609334 [X0 Z1 Z3] +
-0.03355113608569815 [X0 X2] +
0.038058083049884846 [X0 X2 X3] +
-0.038058083049884846 [X0 Y2 Y3] +
0.033078463224609334 [X0 Z2 Z3] +
0.03355113608569815 [X0 X3] +
0.005545747540744297 [Y0 Y1] +
0.03355113608569815 [Y0 Z1 Y2 Z3] +
-0.03355113608569815 [Y0 Z1 Z2 Y3] +
0.038058083049884846 [Y0 X2 Y3] +
-0.03355113608569815 [Y0 Y2] +
0.038058083049884846 [Y0 Y2 X3] +
0.03355113608569815 [Y0 Y3] +
-0.017834277651071245 [Z0] +
-0.03364279215972457 [Z0 X1 X2 Z3] +
-0.03536722155262414 [Z0 X1 Z2] +
0.03364279215972457 [Z0 X1 Z2 X3] +
-0.03536722155262414 [Z0 X1 Z3] +
-0.03364279215972457 [Z0 Y1 Y2 Z3] +
0.03364279215972457 [Z0 Y1 Z2 Y3] +
0.6547228540015417 [Z0 Z1] +
-0.0011411227328337408 [Z0 Z1 X2] +
-0.009390698830211788 [Z0 Z1 X2 Z3] +
0

# And LiH

In [24]:
qubit_transf = 'jw' # Jordan-Wigner transformations
lih = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(lih)

-3.934441956757915 [] +
-0.007923321157849954 [X0 X1 Y2 Y3] +
-0.0034145323580160323 [X0 X1 Y2 Z3 Z4 Y5] +
-0.0027468613203169424 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0034145323580160323 [X0 X1 X3 X4] +
-0.0027468613203169424 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.004864778381761675 [X0 X1 Y4 Y5] +
-0.0022963165872997085 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.002296316587299708 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.002472706168385288 [X0 X1 Y6 Y7] +
-0.0024727061683852886 [X0 X1 Y8 Y9] +
-0.0017744350099502914 [X0 X1 Y10 Y11] +
0.007923321157849954 [X0 Y1 Y2 X3] +
0.0034145323580160323 [X0 Y1 Y2 Z3 Z4 X5] +
0.0027468613203169424 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0034145323580160323 [X0 Y1 Y3 X4] +
-0.0027468613203169424 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.004864778381761675 [X0 Y1 Y4 X5] +
0.0022963165872997085 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.002296316587299708 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.002472706168385288 [X0 Y1 Y6 X7] +
0.0024727061683852886 [X0 Y1 Y8 X9] +
0.

In [23]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(lih, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -3.7779692011782107 [] +
0.04069606505875718 [X0] +
-0.002366478148278278 [X0 X1 Y2 Y3] +
0.0072370922090284825 [X0 X1 Y2 Z3 Z6 Y7] +
0.0016354047160369379 [X0 X1 Z2 X3 Z4 Z5 Z7] +
-0.0031369437172317546 [X0 X1 Z2 Z4 Z5 Z6 X7] +
0.0072370922090284825 [X0 X1 X3 X6] +
-0.0016354047160369379 [X0 X1 X3 Z6 Z7] +
0.0067955266824246885 [X0 X1 X4] +
0.00679552668242469 [X0 X1 X5] +
-0.030603911418853515 [X0 X1 Y6 Y7] +
0.0031369437172317546 [X0 X1 X7] +
0.001635404716036938 [X0 Y1 Y2] +
0.002366478148278278 [X0 Y1 Y2 X3] +
-0.001635404716036938 [X0 Y1 Y2 Z3 Z4 Z5 Z7] +
-0.0072370922090284825 [X0 Y1 Y2 Z3 Z6 X7] +
0.0016354047160369379 [X0 Y1 Z2 Y3 Z4 Z5 Z7] +
-0.003136943717231754 [X0 Y1 Z2 Z3 Y6] +
0.003136943717231754 [X0 Y1 Z2 Z4 Z5 Y6 Z7] +
-0.0031369437172317546 [X0 Y1 Z2 Z4 Z5 Z6 Y7] +
0.0072370922090284825 [X0 Y1 Y3 X6] +
-0.0016354047160369379 [X0 Y1 Y3 Z6 Z7] +
0.0067955266824246885 [X0 Y1 Y4] +
0.00679552668242469 [X0 Y1 Y5] +
0.030603911418853515 [X0 Y1 Y